# Coding an LLM architecture

**Descrição**  
Este notebook monta o **esqueleto de uma arquitetura GPT (LLM decoder-only)** em PyTorch, com foco no **fluxo de dados e nos formatos (shapes)**. Para isso, ele centraliza os hiperparâmetros em um dicionário de configuração, valida essa configuração com uma função utilitária e implementa versões *placeholder* (dummy) de componentes essenciais (bloco Transformer e LayerNorm). A ideia é deixar o pipeline completo pronto para, depois, substituir os “dummies” pelas implementações reais. :contentReference[oaicite:0]{index=0}:contentReference[oaicite:1]{index=1}

**Objetivo**  
- Definir um **contrato claro de configuração** (hiperparâmetros obrigatórios e coerência).  
- Garantir que a arquitetura esteja **estruturada como um GPT** (embeddings → blocos Transformer → normalização → projeção para o vocabulário), mesmo antes da atenção/MLP reais.  
- Permitir testar cedo o `forward` e os shapes de entrada/saída, reduzindo bugs quando os módulos verdadeiros forem adicionados. :contentReference[oaicite:2]{index=2}

**Funcionamento**  
1. **Configuração (`GPT_CONFIG_124M`)**: define `vocab_size`, `context_length`, `emb_dim`, `n_heads`, `n_layers`, `drop_rate`, `qkv_bias`.  
2. **Validação (`validate_gpt_config`)**: checa presença de chaves, tipos e valores aceitáveis antes de construir o modelo.  
3. **Componentes dummy**:  
   - `DummyTransformerBlock`: mantém a interface, mas retorna a entrada sem mudanças.  
   - `DummyLayerNorm`: mantém a interface, mas não normaliza (retorna a entrada).  
4. **Modelo (`DummyGPTModel`)**:  
   - Recebe `in_idx` com shape **(batch_size, seq_len)** e valida `seq_len <= context_length`.  
   - Calcula **token embeddings** e **positional embeddings**, soma e aplica dropout.  
   - Passa pela pilha de “blocos Transformer” (dummy) e por uma normalização final (dummy).  
   - Projeta para logits via `out_head`, retornando **(batch_size, seq_len, vocab_size)**. :contentReference[oaicite:3]{index=3}


![O gato sobe no tapete](../../imagens/cap04/01_building_blocks.png)

In [1]:
from collections.abc import Mapping
from typing import Any

import torch
import torch.nn as nn
from torch import Tensor

## Configuração do modelo (hiperparâmetros)

Este dicionário reúne os **hiperparâmetros** que definem o “tamanho” e o comportamento básico do GPT (arquitetura e regularização). A ideia é centralizar tudo em um único lugar para que o resto do código (embeddings, blocos Transformer, etc.) consiga ler esses valores de forma consistente.

* **`vocab_size`**: quantidade de tokens possíveis no vocabulário (define o tamanho da tabela de embeddings e a dimensão de saída da *head* de logits).
* **`context_length`**: tamanho máximo de sequência (quantos tokens o modelo consegue “enxergar” de uma vez), usado no embedding posicional.
* **`emb_dim`**: dimensão dos vetores de embedding (também chamada de `d_model`), que é a “largura” das representações internas.
* **`n_heads`**: número de cabeças de atenção em *multi-head attention* (em implementações reais, `emb_dim` costuma ser divisível por `n_heads`).
* **`n_layers`**: número de blocos Transformer empilhados (profundidade do modelo).
* **`drop_rate`**: taxa de dropout para regularização (ajuda a reduzir overfitting durante o treino).
* **`qkv_bias`**: indica se as projeções lineares de **Query/Key/Value** usam termo de viés (bias) ou não.


In [2]:
GPT_CONFIG_124M: dict[str, Any] = {
    "vocab_size": 50257,  # Tamanho do vocabulário
    "context_length": 1024,  # Comprimento do contexto
    "emb_dim": 768,  # Dimensão do embedding
    "n_heads": 12,  # Número de cabeças de atenção
    "n_layers": 12,  # Número de camadas
    "drop_rate": 0.1,  # Taxa de dropout
    "qkv_bias": False,  # Viés em Query-Key-Value (QKV)
}

## Função de validação da configuração (`validate_gpt_config`)

Este trecho cria uma função utilitária para **verificar se o dicionário `cfg` está completo e coerente** antes de construir o modelo. Isso evita erros difíceis de debugar mais adiante (por exemplo, `KeyError` no meio do `forward` ou shapes incompatíveis) e garante que hiperparâmetros essenciais estejam presentes, com tipos e faixas aceitáveis.

* **Assinatura e objetivo**

  * `cfg: Mapping[str, Any]`: aceita qualquer “dicionário-like” (dict, `Mapping`, configs imutáveis etc.).
  * Retorna `None`: a função apenas **valida**; se algo estiver errado, ela **interrompe** com exceção.

* **`required_keys`: contrato do que o modelo espera**

  * Define um “schema” mínimo: cada chave obrigatória e o(s) tipo(s) aceito(s).
  * `drop_rate` aceita `(float, int)` porque é comum o usuário passar `0` ou `1` como inteiros.

* **Checagem de chaves ausentes**

  * `missing = [...]` coleta tudo que falta.
  * Se houver faltas, levanta `KeyError` com uma mensagem clara, indicando exatamente quais chaves precisam ser adicionadas.

* **Checagem de tipos**

  * Itera por `required_keys.items()` e valida com `isinstance`.
  * Se algum tipo estiver incorreto, levanta `TypeError` dizendo:

    * qual chave falhou,
    * qual tipo era esperado,
    * qual tipo foi recebido.

* **Validações de domínio (valores válidos)**

  * Garante que parâmetros que definem dimensões e contagens sejam **positivos**:

    * `vocab_size`, `context_length`, `emb_dim`, `n_heads`, `n_layers` > 0
  * Restringe `drop_rate` ao intervalo **[0.0, 1.0]**, já que dropout é uma probabilidade/taxa.
  * Cada regra levanta `ValueError` com uma mensagem direta, facilitando a correção.

Em resumo, essa função funciona como uma “barreira de segurança”: **se a configuração passar por aqui, o restante do código pode assumir que `cfg` tem o formato esperado**.


In [3]:
def validate_gpt_config(cfg: Mapping[str, Any]) -> None:
    """
    Valida um dicionário de configuração para um GPT-like model.

    Parâmetros
    ----------
    cfg : Mapping[str, Any]
        Dicionário (ou mapping) com as chaves esperadas pelo modelo.

    Retorno
    -------
    None

    Exceções
    --------
    Levanta KeyError se faltar alguma chave obrigatória.
    Levanta TypeError/ValueError se algum valor for inválido.
    """
    required_keys = {
        "vocab_size": int,
        "context_length": int,
        "emb_dim": int,
        "n_heads": int,
        "n_layers": int,
        "drop_rate": (float, int),
        "qkv_bias": bool,
    }

    missing = [k for k in required_keys if k not in cfg]
    if missing:
        raise KeyError(f"Config incompleta. Faltando chaves: {missing}")

    for k, expected_type in required_keys.items():
        if not isinstance(cfg[k], expected_type):
            raise TypeError(
                f"cfg['{k}'] deve ser do tipo {expected_type}, mas recebeu {type(cfg[k])}."
            )

    if int(cfg["vocab_size"]) <= 0:
        raise ValueError("cfg['vocab_size'] deve ser > 0.")
    if int(cfg["context_length"]) <= 0:
        raise ValueError("cfg['context_length'] deve ser > 0.")
    if int(cfg["emb_dim"]) <= 0:
        raise ValueError("cfg['emb_dim'] deve ser > 0.")
    if int(cfg["n_heads"]) <= 0:
        raise ValueError("cfg['n_heads'] deve ser > 0.")
    if int(cfg["n_layers"]) <= 0:
        raise ValueError("cfg['n_layers'] deve ser > 0.")
    if not (0.0 <= float(cfg["drop_rate"]) <= 1.0):
        raise ValueError("cfg['drop_rate'] deve estar no intervalo [0.0, 1.0].")

## Classe `DummyTransformerBlock` (bloco Transformer “falso”)

Este trecho define um **bloco Transformer placeholder** que mantém a *mesma interface* (construtor + `forward`) de um bloco real, mas **não aplica nenhuma transformação** nos dados. Ele é útil para montar a “carcaça” da arquitetura do GPT (empilhamento de blocos, fluxo do `forward`, shapes esperados) antes de implementar atenção, MLP, residuais e normalizações de verdade.

* **Por que existe?**

  * Permite testar o pipeline completo do modelo (embeddings → blocos → head de saída) sem depender das partes complexas ainda não implementadas.
  * Ajuda a garantir que os tensores tenham os formatos corretos e que o código esteja bem organizado para substituição posterior.

* **`__init__(cfg)`**

  * Chama `super().__init__()` para inicializar corretamente o `nn.Module`.
  * Executa `validate_gpt_config(cfg)` para garantir que a configuração possui as chaves e valores esperados, evitando erros silenciosos ou inconsistências.

* **`forward(x)`**

  * Recebe `x` no formato típico **(batch, seq_len, emb_dim)**.
  * Retorna **exatamente o mesmo tensor** (`return x`), sem atenção, sem MLP, sem dropout, sem conexões residuais.
  * Mantém a assinatura e o comportamento mínimo para que `nn.Sequential` consiga encadear vários “blocos” de forma idêntica a um Transformer real.


In [4]:
class DummyTransformerBlock(nn.Module):
    """
    Placeholder de um TransformerBlock.

    Observação:
    ----------
    Esta implementação não faz nada além de retornar a entrada.
    Ela existe apenas para manter a estrutura do modelo enquanto
    os blocos reais são implementados nas próximas seções.
    """

    def __init__(self, cfg: Mapping[str, Any]) -> None:
        super().__init__()
        validate_gpt_config(cfg)

    def forward(self, x: Tensor) -> Tensor:
        """
        Retorna a entrada inalterada.

        Parâmetros
        ----------
        x : Tensor
            Tensor de entrada no formato (batch, seq_len, emb_dim).

        Retorno
        -------
        Tensor
            O mesmo tensor de entrada.
        """
        return x

## Classe `DummyLayerNorm` (LayerNorm “falso”)

Este trecho define uma **versão placeholder de `LayerNorm`**: ela mantém a mesma estrutura e parâmetros esperados por uma normalização real, mas **não realiza nenhuma normalização** (apenas devolve a entrada). O objetivo é permitir que a arquitetura completa do GPT seja montada e testada desde cedo, deixando a implementação correta da normalização para uma etapa posterior.

* **Por que mimetizar o `LayerNorm`?**

  * Em um Transformer real, `LayerNorm` é parte fundamental da estabilidade do treinamento.
  * Aqui, a classe existe para preservar o “formato” do código (mesma API), facilitando substituir `DummyLayerNorm` por uma implementação real sem mudar o restante do modelo.

* **`__init__(normalized_shape, eps=1e-5)`**

  * Chama `super().__init__()` para inicializar corretamente o `nn.Module`.
  * Valida `normalized_shape`:

    * precisa ser `int` e **maior que 0**, pois representa a dimensão que seria normalizada (tipicamente `emb_dim`).
  * Armazena:

    * `self.normalized_shape` (metadado/compatibilidade)
    * `self.eps` como `float` (em LayerNorm real, `eps` evita divisão por zero).

* **`forward(x)`**

  * Recebe um tensor `x` (sem impor shape específico aqui).
  * Retorna `x` **inalterado**, sem calcular média, variância ou aplicar escala/viés.
  * Mantém a assinatura para que o restante do pipeline funcione como se houvesse uma normalização real.


In [5]:
class DummyLayerNorm(nn.Module):
    """
    Placeholder de LayerNorm.

    Observação:
    ----------
    Esta implementação não normaliza; apenas retorna a entrada.
    Ela mimetiza a interface para facilitar a troca posterior.
    """

    def __init__(self, normalized_shape: int, eps: float = 1e-5) -> None:
        super().__init__()
        if not isinstance(normalized_shape, int) or normalized_shape <= 0:
            raise ValueError("normalized_shape deve ser um int > 0.")
        self.normalized_shape = normalized_shape
        self.eps = float(eps)

    def forward(self, x: Tensor) -> Tensor:
        """
        Retorna a entrada inalterada.

        Parâmetros
        ----------
        x : Tensor
            Tensor de entrada.

        Retorno
        -------
        Tensor
            O mesmo tensor de entrada.
        """
        return x

## Classe `DummyGPTModel` (esqueleto de um GPT)

Este trecho implementa um **modelo GPT simplificado**, montando a estrutura principal do pipeline (embeddings → blocos Transformer → normalização → projeção para vocabulário), mas usando **componentes “dummy”** (`DummyTransformerBlock` e `DummyLayerNorm`) que ainda não fazem o trabalho real. A intenção é ter uma arquitetura funcional em termos de *shapes* e fluxo de dados, pronta para receber as implementações completas nas próximas seções.

---

### Visão geral dos componentes

* **Token Embedding (`tok_emb`)**
  Converte cada índice de token em um vetor denso de dimensão `emb_dim`.
  Saída típica: `(B, T, C)`.

* **Positional Embedding (`pos_emb`)**
  Cria embeddings para posições `0..T-1`, permitindo ao modelo diferenciar a ordem dos tokens.
  Saída típica: `(T, C)`.

* **Dropout (`drop_emb`)**
  Regularização aplicada após somar token + posição (com taxa `drop_rate`).

* **Pilha de blocos Transformer (`trf_blocks`)**
  Aqui é uma sequência de `n_layers` blocos *placeholder*, construída com `nn.Sequential`, mantendo a mesma estrutura de um GPT real.

* **Normalização final (`final_norm`)**
  Placeholder de LayerNorm aplicado antes da camada de saída (em GPTs reais isso ajuda estabilidade).

* **Cabeça de saída (`out_head`)**
  Projeção linear de `emb_dim` para `vocab_size`, gerando **logits** para cada token do vocabulário em cada posição da sequência.

---

#### `__init__(cfg)`: construção do modelo a partir da configuração

* **Validação do `cfg`**

  * `validate_gpt_config(cfg)` garante que as chaves existem, tipos estão corretos e valores fazem sentido.

* **Leitura e normalização dos hiperparâmetros**

  * Converte explicitamente para `int/float` para evitar surpresas com tipos.

* **Criação das camadas**

  * `nn.Embedding(vocab_size, emb_dim)`: tabela de embeddings de tokens.
  * `nn.Embedding(context_length, emb_dim)`: tabela de embeddings posicionais até o máximo de contexto.
  * `nn.Dropout(drop_rate)`: dropout aplicado no embedding somado.
  * `nn.Sequential(*[...])`: empilha `n_layers` blocos.
  * `nn.Linear(emb_dim, vocab_size, bias=False)`: transforma vetores internos em logits do vocabulário.

---

#### `forward(in_idx)`: fluxo do dado no modelo

* **Validações de entrada**

  * Garante que `in_idx` é um `Tensor`.
  * Exige que seja 2D: `(batch_size, seq_len)`.
  * Verifica se `seq_len` não excede `context_length` (porque o embedding posicional foi criado com esse limite).

* **Embeddings**

  * `tok_embeds = tok_emb(in_idx)` → `(B, T, C)`
    Cada token vira um vetor.
  * `pos_ids = arange(seq_len)` e `pos_embeds = pos_emb(pos_ids)` → `(T, C)`
    Cada posição recebe um vetor.

* **Combinação token + posição**

  * `x = tok_embeds + pos_embeds`
    O PyTorch faz *broadcasting*: `(B, T, C) + (T, C) → (B, T, C)`.
    Isso injeta informação de ordem na representação.

* **Aplicação das “partes do Transformer”**

  * `x = drop_emb(x)` aplica dropout.
  * `x = trf_blocks(x)` passa por `n_layers` blocos (aqui não alteram nada, mas preservam o pipeline).
  * `x = final_norm(x)` normalização final (placeholder).

* **Geração dos logits**

  * `logits = out_head(x)` → `(B, T, vocab_size)`
    Para cada posição `T` e item do batch `B`, o modelo produz uma distribuição (logits) sobre todos os tokens do vocabulário.

---

#### Resultado final

Ao final, a classe entrega um modelo que **já produz logits com o shape correto** para geração de texto e treino (cross-entropy), mesmo antes da implementação real dos blocos Transformer e da LayerNorm. Isso facilita construir e testar a arquitetura por etapas, substituindo os placeholders gradualmente.


In [6]:
class DummyGPTModel(nn.Module):
    """
    Arquitetura GPT simplificada (com blocos e LayerNorm 'dummy').

    Componentes:
    -----------
    - Token embedding
    - Positional embedding
    - Dropout
    - Pilha de blocos Transformer (placeholder)
    - LayerNorm final (placeholder)
    - Cabeça linear para logits no vocabulário
    """

    def __init__(self, cfg: Mapping[str, Any]) -> None:
        super().__init__()
        validate_gpt_config(cfg)

        vocab_size = int(cfg["vocab_size"])
        emb_dim = int(cfg["emb_dim"])
        context_length = int(cfg["context_length"])
        drop_rate = float(cfg["drop_rate"])
        n_layers = int(cfg["n_layers"])

        self.vocab_size = vocab_size
        self.emb_dim = emb_dim
        self.context_length = context_length

        self.tok_emb = nn.Embedding(vocab_size, emb_dim)
        self.pos_emb = nn.Embedding(context_length, emb_dim)
        self.drop_emb = nn.Dropout(drop_rate)

        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(n_layers)]
        )
        self.final_norm = DummyLayerNorm(emb_dim)
        self.out_head = nn.Linear(emb_dim, vocab_size, bias=False)

    def forward(self, in_idx: Tensor) -> Tensor:
        """
        Executa o forward pass.

        Parâmetros
        ----------
        in_idx : Tensor
            Tensor de índices de tokens no formato (batch_size, seq_len),
            tipicamente dtype torch.long.

        Retorno
        -------
        Tensor
            Logits no formato (batch_size, seq_len, vocab_size).

        Exceções
        --------
        Levanta ValueError se seq_len > cfg["context_length"].
        Levanta TypeError se o input não for um Tensor 2D.
        """
        if not isinstance(in_idx, Tensor):
            raise TypeError("in_idx deve ser um torch.Tensor.")
        if in_idx.ndim != 2:
            raise TypeError("in_idx deve ter 2 dimensões: (batch_size, seq_len).")

        batch_size, seq_len = in_idx.shape
        if seq_len > self.context_length:
            raise ValueError(
                f"seq_len ({seq_len}) excede context_length ({self.context_length})."
            )

        tok_embeds = self.tok_emb(in_idx)  # (B, T, C)
        pos_ids = torch.arange(seq_len, device=in_idx.device)
        pos_embeds = self.pos_emb(pos_ids)  # (T, C)

        x = tok_embeds + pos_embeds  # broadcast: (B, T, C) + (T, C)
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)

        logits = self.out_head(x)  # (B, T, vocab_size)
        return logits